In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import  TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
credits = pd.read_csv('tmdb_5000_credits.csv')
movies = pd.read_csv('tmdb_5000_movies.csv')

In [3]:
credits.columns

Index(['movie_id', 'title', 'cast', 'crew'], dtype='object')

In [4]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [5]:
credits.columns = ['id', 'title', 'cast', 'crew']
movies = movies.merge(credits, on='id')

In [6]:
movies['overview'].head(5)

0    In the 22nd century, a paraplegic Marine is di...
1    Captain Barbossa, long believed to be dead, ha...
2    A cryptic message from Bond’s past sends him o...
3    Following the death of District Attorney Harve...
4    John Carter is a war-weary, former military ca...
Name: overview, dtype: object

In [7]:
movies['overview'] = movies['overview'].fillna('')

In [8]:
def create_join(x):
    return ''.join(x['keywords']) + '' + ''.join(x['genres']) + '' + ''.join(x['overview'])
movies['join'] = movies.apply(create_join, axis=1)

In [9]:
tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrices = tfidf.fit_transform(movies['join'])
tfidf_matrices.shape

(4803, 32768)

In [10]:
cosine_similarity = linear_kernel(tfidf_matrices,tfidf_matrices)

In [11]:
indices = pd.Series(movies.index, index=credits['title']).drop_duplicates()

In [12]:
def Recommendation_get(title, cosine_similarity = cosine_similarity):
    idx = indices[title]
    similar_scores = list(enumerate(cosine_similarity[idx]))
    similar_scores = sorted(similar_scores, key = lambda x: x[1], reverse = True)
    similar_scores = similar_scores[1:21]
    movie_indices = [i[0] for i in similar_scores]
    return credits['title'].iloc[movie_indices]

In [14]:
title = input(str('Enter a movie title that you watched recently: '))
print("\t")
print("If you like that then you might like these .........")
Recommendation_get(title, cosine_similarity)

Enter a movie title that you watched recently: Skyfall
	
If you like that then you might like these .........


2                               Spectre
1343              Never Say Never Again
4339                             Dr. No
4071              From Russia with Love
3143                You Only Live Twice
3285                           Restless
3343                   Live and Let Die
2675               The Spy Who Loved Me
11                    Quantum of Solace
3251    On Her Majesty's Secret Service
1743                          Octopussy
147                     Die Another Day
1200               The Living Daylights
3336               Diamonds Are Forever
1713                 For Your Eyes Only
425                 Mission: Impossible
2428                  Brooklyn's Finest
139             Mission: Impossible III
1077                     Johnny English
3351        The Man with the Golden Gun
Name: title, dtype: object